# INF264 - Project 1
`@authors` Kristin Loka Øydna and Silje Folkestad

## Overview
0. [Imports](#Imports)
1. [Reading Dataset](#Reading-Dataset)
    1. [Loading Dataset](#Loading-Dataset)
    2. [Clean Up](#Clean-Up)
2. [Task 1.1 / 1.2 / 1.3](#Task-1.1-/-1.2-/-1.3)
    1. [Node Class](#Node-class)
    2. [Decision Tree Class](#Decision-tree-class)
3. [Train_test_split](#Train_test_split)
4. [Task 1.4: Evaluate Algorithm](#Task-1.4)
    1. [Model Selection](#Model-Selection)
    2. [Model Evaluation](#Model-Evaluation)
5. [Task 1.5: Comparing to sklearn.tree.DecisionTreeClassifier](#Task-1.5)

## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

## Reading Dataset

### Loading Datasets

In [2]:
data = pd.read_csv("magic04.data", header=None, sep=",")

### Clean Up

In [3]:
'''Export the dataframe to numpy'''
X = data.iloc[:, :-1].to_numpy().astype(float)
y_temp = data.iloc[:, -1].to_numpy()

In [4]:
'''Checking how many distinct values there are in y_temp'''
np.unique(y_temp)

array(['g', 'h'], dtype=object)

In [5]:
'''Changing the values in y_temp to 1 and 0'''
y = np.zeros(shape=y_temp.shape)
y[y_temp == 'h']=1.0

data = np.concatenate((X, np.expand_dims(y,axis=1)), axis = 1)
data = pd.DataFrame(data)

In [6]:
'''Checking if there are any missing values'''

# Replacing the missing values with np.NaN
data.replace(r'^\s*$', np.NaN, regex=True, inplace = True)

# Checking if there are any np.NaN
data.isnull().values.any()

False

## Task 1.1 / 1.2 / 1.3
### Node Class

In [7]:
class Node():
    def __init__(self, dominant_label = None, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        '''Constructor'''
        self.dominant_label = dominant_label
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

### Decision Tree Class

In [8]:
class DecisionTree():
    def __init__(self,  impurity_measure):
        '''Constructor'''
        self.root = None
        self.impurity_measure = impurity_measure
        
    def build_tree(self, dataset):
        '''Building the tree'''
        X = dataset[:, :-1]
        y = dataset[:, -1]
    
        num_rows, num_feat = np.shape(X)

        labels, counts = np.unique(y, return_counts=True)        
        freq_label = labels[np.argmax(counts)]
        
        if labels.shape[0]==1:
            return Node(value = y[0])
        
        elif np.unique(X, axis = 0).shape[0] == 1:
            return Node(value = freq_label)
        
        else:
            best_split = self.get_best_split(dataset, num_feat)
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"]) #Recursive 
                right_subtree = self.build_tree(best_split["dataset_right"]) #Recursive
                return Node(feature_index = best_split["feature_index"], threshold = best_split["threshold"], 
                            left = left_subtree, right = right_subtree, info_gain = best_split["info_gain"], 
                            dominant_label = freq_label)
    
    def get_best_split(self, dataset, num_feat):
        '''Finds the best split'''
        
        best_split={}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_feat):
            feature_values = dataset[:, feature_index]
            threshold =  np.mean(feature_values)
            dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
            if len(dataset_left)>0 and len(dataset_right)>0:
                y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                curr_info_gain = self.information_gain(y, left_y, right_y, self.impurity_measure)
                if curr_info_gain > max_info_gain:
                    best_split["feature_index"] = feature_index
                    best_split["threshold"] = threshold
                    best_split["dataset_left"] = dataset_left
                    best_split["dataset_right"] = dataset_right
                    best_split["info_gain"] = curr_info_gain
                    max_info_gain = curr_info_gain
        return best_split
        
    def split(self, dataset, feature_index, threshold):
        '''Splitting data'''
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    # Task 1.1/1.2
    def information_gain(self, parent, l_child, r_child, impurity_measure):
        '''Calculating information gain'''
        impurity_measure = self.impurity_measure
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if impurity_measure == 'gini':
            gain = 1-(self.gini(parent) - (weight_l*self.gini(l_child) + weight_r*self.gini(r_child)))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain    

    def entropy(self, y):
        '''Calculating entropy'''
        class_labels = np.unique(y)
        entropy = 0
        for label in class_labels:
            prob_label = len(y[y == label])/len(y)
            entropy += -prob_label*np.log2(prob_label)
        return entropy

    # Task 1.2
    def gini(self, y):
        '''Calculating gini'''
        class_labels = np.unique(y)
        gini = 0
        for label in class_labels:
            prob_label = len(y[y == label])/len(y)
            gini += prob_label**2
        return gini
    
    def fit(self, X, Y):
        '''Training the tree'''
        dataset = np.concatenate((X, Y), axis = 1)
        self.root = self.build_tree(dataset)
            
    def predicts(self, X):
        '''Predicting the new dataset'''
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
        
    def make_prediction(self, x, tree):
        '''Predicting a new data point'''
        if tree.value is not None: 
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
        
    #Task 1.3
    def reduced_error_pruning(self, node, pruning_dataset):
        if node.value is not None:
            return np.sum(node.value != pruning_dataset[:, -1])

        split_left = pruning_dataset[:,node.feature_index] <= node.threshold
        split_right = ~split_left #the opposite of split_left
        pruning_dataset_left = pruning_dataset[split_left, :]
        pruning_dataset_right = pruning_dataset[split_right, :]
        
        
        error_left = self.reduced_error_pruning(node.left, pruning_dataset_left)
        error_right = self.reduced_error_pruning(node.right, pruning_dataset_right)
        
        error_without_pruning = error_left + error_right
        
        error_with_pruning = np.sum(node.dominant_label != pruning_dataset[:, -1])
        
        if error_without_pruning >= error_with_pruning:
            node.left = None
            node.right = None
            node.value = node.dominant_label
            return error_with_pruning
        else:
            return error_without_pruning

## Train_test_split

In [9]:
class Train_test_split():
    def __init__(self, dataset):
        self.dataset = dataset
        
    def split_into_training_data(self, dataset):
        X = dataset.iloc[:, :-1].values
        Y = dataset.iloc[:, -1].values.reshape(-1, 1)

        X_train, X_val_test, Y_train, Y_val_test = train_test_split(X, Y, test_size = 0.4, shuffle = True, random_state = 42)
        X_val, X_test, Y_val, Y_test = train_test_split(X_val_test, Y_val_test, test_size = 0.5, shuffle = True, random_state = 42)
        return (X_train, Y_train, X_val, Y_val, X_test, Y_test)
    
    def split_into_pruning_data(self, X_train, Y_train):
        X_after_train, X_prune, Y_after_train, Y_prune = train_test_split(X_train, Y_train, test_size = 0.5, shuffle = True, random_state = 42)
        dataset_train = np.concatenate((X_after_train, Y_after_train), axis = 1)
        dataset_prune = np.concatenate((X_prune, Y_prune), axis = 1)
        return (dataset_train, dataset_prune, X_after_train, Y_after_train)

## Task 1.4

### Evaluate Algorithm

#### Model Selection

In [10]:
splitter = Train_test_split(data)
X_train, Y_train, X_val, Y_val, X_test, Y_test = splitter.split_into_training_data(data)
dataset_train, dataset_prune, X_after_train, Y_after_train = splitter.split_into_pruning_data(X_train, Y_train)

In [11]:
'''Entropy accuracy'''
classifier = DecisionTree(impurity_measure = "entropy")

classifier.fit(X_train, Y_train)
Y_pred = classifier.predicts(X_val)
entropy_without_pruning_acc = accuracy_score(Y_val, Y_pred)
print("Accuracy score with entropy:", entropy_without_pruning_acc)

Accuracy score with entropy: 0.7957413249211357


In [12]:
'''Entropy accuracy with pruning'''
classifier = DecisionTree(impurity_measure = "entropy")

classifier.fit(X_after_train, Y_after_train)
classifier.reduced_error_pruning(classifier.root, dataset_prune)
Y_pred = classifier.predicts(X_val)
entropy_with_pruning_acc = accuracy_score(Y_val, Y_pred)
print("Accuracy score with entropy and pruning:", entropy_with_pruning_acc)

Accuracy score with entropy and pruning: 0.8288643533123028


In [13]:
'''Gini index accuracy'''
classifier = DecisionTree(impurity_measure = "gini")

classifier.fit(X_train, Y_train)
Y_pred = classifier.predicts(X_val)
gini_without_pruning_acc = accuracy_score(Y_val, Y_pred)
print("Accuracy score with gini index:", gini_without_pruning_acc)

Accuracy score with gini index: 0.7915352260778128


In [14]:
'''Gini index with pruning'''
classifier = DecisionTree(impurity_measure = "gini")

classifier.fit(X_after_train, Y_after_train)
classifier.reduced_error_pruning(classifier.root, dataset_prune)
Y_pred = classifier.predicts(X_val)
gini_with_pruning_acc = accuracy_score(Y_val, Y_pred)
print("Accuracy score with gini index and pruning:", gini_with_pruning_acc)

Accuracy score with gini index and pruning: 0.8264984227129337


In [15]:
'''Checks the highest accuracy score between all of our models'''

highest_acc = max(entropy_without_pruning_acc, entropy_with_pruning_acc, gini_without_pruning_acc, gini_with_pruning_acc)

acc={"entropy":entropy_without_pruning_acc, "entropy with pruning":entropy_with_pruning_acc, 
     "gini index":gini_without_pruning_acc, "gini index with pruning":gini_with_pruning_acc}

print("The model that performes the best is",max(acc, key=acc.get))

The model that performes the best is entropy with pruning


Since pruning with entropy gives the best accuracy on the validation dataset, we will use that model to evaluate on test data.

#### Model Evaluation

In [16]:
classifier = DecisionTree(impurity_measure = "entropy")

classifier.fit(X_after_train, Y_after_train)
classifier.reduced_error_pruning(classifier.root, dataset_prune)
Y_pred = classifier.predicts(X_test)
entropy_with_pruning_acc = accuracy_score(Y_test, Y_pred)
print("Accuracy score with entropy and pruning on test set:", entropy_with_pruning_acc)

Accuracy score with entropy and pruning on test set: 0.8378023133543638


## Task 1.5

### Comparing to sklearn.tree.DecisionTreeClassifier

In [17]:
'''Calculating accuracy with sklearn.tree.DecisionTreeClassifer'''
clf = DecisionTreeClassifier(criterion="entropy", random_state=42)
clf.fit(X_train, Y_train)
Y_predcls = clf.predict(X_test)

print("Accuracy score when using existing decision tree implementation:", accuracy_score(Y_test, Y_predcls))

Accuracy score when using existing decision tree implementation: 0.8194006309148265


The accuracy score from our algorithm is better than the accuracy score when using existing decision tree implementation.